In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential

In [ ]:
train = pd.read_csv('../input/dataset/dataset/Train.csv')
test = pd.read_csv('../input/dataset/dataset/Test.csv')

In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:


from PIL import Image
img = Image.open('../input/dataset/dataset/train images/Images/id_02amazy34fgh2.jpg')
nparr  = np.asarray(img)



In [ ]:
nparr.shape

In [ ]:


train.iloc[2]['Image_id']



In [ ]:
from sklearn.model_selection import train_test_split

train,validate = train_test_split(train,test_size=0.2)

new_img_size = (224,224)



In [ ]:
validate.Label.value_counts()

In [ ]:
images_train=[]
img = Image.open('../input/dataset/dataset/train images/Images/'+str(train.iloc[0]['Image_id']))
img=img.resize(new_img_size)
images_train = np.array([np.asarray(img)])

for i in range(1,len(train)):
    img = Image.open(f'../input/dataset/dataset/train images/Images/'+str(train.iloc[i]['Image_id']))
    img = img.resize(new_img_size)
    images_train = np.concatenate((images_train,np.array([np.asarray(img)])),axis=0)

In [ ]:
images_train[0]

In [ ]:
img = Image.open('../input/dataset/dataset/train images/Images/'+str(validate.iloc[0]["Image_id"]))
img=img.resize(new_img_size)
images_validate = np.array([np.asarray(img)])
for i in range(1,validate.shape[0]):
    img = Image.open('../input/dataset/dataset/train images/Images/'+str(validate.iloc[i]["Image_id"]))
    img = img.resize(new_img_size)
    images_validate = np.concatenate((images_validate,np.array([np.asarray(img)])),axis=0)

In [ ]:


img = Image.open(f'../input/dataset/dataset/train images/Images/'+str(test.iloc[0]["Image_id"]))
img=img.resize(new_img_size)
images_test = np.array([np.asarray(img)])
for i in range(1,test.shape[0]):
    img = Image.open('../input/dataset/dataset/train images/Images/'+str(test.iloc[i]["Image_id"]))
    img = img.resize(new_img_size)
    images_test = np.concatenate((images_test,np.array([np.asarray(img)])),axis=0)



In [ ]:
print("Training set is of shape : ",images_train.shape)
print("Validation set is of shape : ",images_validate.shape)
print("Test set is of shape : ",images_test.shape)


# **Model**

In [ ]:
model = Sequential([
    layers.Rescaling(1./255.,input_shape=(224,224,3)),
    layers.Conv2D(8,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid'),   
])
model.build()


In [ ]:


model.compile(
    optimizer='adam',
    loss = tf.losses.BinaryCrossentropy(),
    metrics=['binary_accuracy']
)



In [ ]:
batch_size = 32
epochs = 15

model.fit(
    images_train,
    train['Label'],
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(images_validate,validate['Label'])
)


In [ ]:


data_augmentaion = Sequential([
    layers.RandomRotation(factor=(-0.2,0.2),seed = 123),
    layers.RandomZoom(0.1),
])



In [ ]:
model = Sequential([
    layers.Rescaling(1./255.,input_shape=(224,224,3)),
    data_augmentaion,
    layers.Conv2D(8,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid'),   
])
model.build()

In [ ]:


model.compile(
    optimizer='adam',
    loss = tf.losses.BinaryCrossentropy(),
    metrics=['binary_accuracy']
)

batch_size = 32
epochs = 100

history = model.fit(
    images_train,
    train['Label'],
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(images_validate,validate['Label'])
)



In [ ]:
train_acc = history.history['binary_accuracy']
validate_acc = history.history['val_binary_accuracy']
epoch_ = range(1,epochs+1)

plt.plot(epoch_,train_acc,'r',label='Training')
plt.plot(epoch_,validate_acc,'b',label='Validation')
plt.legend()
plt.show()


# Prediction time!

In [ ]:


predictions = model.predict(images_test)



In [ ]:


predictions.shape



# **Submission**

In [ ]:


predictions = pd.DataFrame(np.round(predictions,2))
submission = pd.concat([test,predictions],axis=1)



In [ ]:
submission.rename(columns = {0:'Target', 'Image_id':'Image_ID'}, inplace = True)

In [ ]:
submission.tail(5)

In [ ]:
label_encode = {0:"healthy",1:"multiple_diseases",2:"rust",3:"scab"}

In [ ]:


submission.to_csv("submission.csv",index=False)

